In [4]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents, fetch_website_links
import json
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr # oh yeah!


load_dotenv(override=True)

api_key = os.getenv('OLLAMA_API_KEY')

ollama_base_url = os.getenv("OLLAMA_BASE_URL")

sales_pitch_username = os.getenv("SALES_PITCH_USERNAME")
sales_pitch_password = os.getenv("SALES_PITCH_PASSWORD")

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


#Check the base URL
if not ollama_base_url:
    print("No base url was found - please confirm that the lamma base url has been set in the dotenv file")
elif ollama_base_url.strip() != ollama_base_url:
    print("A base url was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("Base url found and looks good so far!")


ollamaClient = OpenAI(base_url=ollama_base_url, api_key=api_key)

ed = fetch_website_contents("https://edwarddonner.com")

link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

brochure_system_prompt = """
You are an assistant who is prolific at pitching ideas to investors. 
You analyze the contents of several relevant pages from a company website and create an excellent pitch for any investor, 
either an angel investor or not, for why they should invest in the idea
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt




def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling gpt-oss:120b-cloud")
    response = ollamaClient.chat.completions.create(
        model="gpt-oss:120b-cloud",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links


def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result


def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


def create_brochure(company_name, url):
    stream = ollamaClient.chat.completions.create(
        model="gpt-oss:120b-cloud",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content
        yield result

business_name_input = gr.Textbox(label="Business name:", info="Enter the business name", lines=3)
business_name_weblink = gr.Textbox(label="Business Web Link", info="Enter the business's website URL", lines=1)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=create_brochure,
    title="Pitch Sales Generator", 
    inputs=[business_name_input, business_name_weblink], 
    outputs=[message_output], 
    examples=[
        ["Splitly", "https://www.gosplitly.com"],
        ["Huggingface", "https://huggingface.co"]
        ], 
    flagging_mode="never"
    )
view.launch(auth=(sales_pitch_username, sales_pitch_password), share=True)

API key found and looks good so far!
Base url found and looks good so far!
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://9e66d241f59e4b7658.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
